## 📦 Instalación de Librerías Necesarias

**Antes de ejecutar este notebook, asegúrate de tener instaladas todas las dependencias.**

### Opción 1: Instalación dentro del notebook
Ejecuta la siguiente celda para instalar las librerías necesarias:

```python
%pip install pandas numpy plotly nbformat
```

### Opción 2: Instalación desde terminal
Si prefieres instalar desde la terminal, ejecuta:

```bash
# PowerShell o CMD
pip install pandas numpy plotly nbformat

# O si usas el proyecto completo con pyproject.toml
pip install -e .[core,notebooks,iot]
```

### Librerías requeridas:
- `pandas`: Manipulación y análisis de datos
- `numpy`: Cálculos numéricos y arrays
- `plotly`: Visualización interactiva (mapas, gráficos en tiempo real)
- `nbformat`: Requerido para renderizar gráficos de Plotly en notebooks

---

In [1]:
# ⚙️ Preparación de entorno y rutas
# Si esta celda tarda demasiado o se cuelga:
# 1) Abre la paleta de comandos (Ctrl+Shift+P)
# 2) "Jupyter: Restart Kernel"
# 3) "Run All Above/Below" o ejecuta desde la primera celda

import sys
from pathlib import Path

# Detectar raíz del repo (buscando pyproject.toml o carpeta src)
_candidates = [Path.cwd(), *Path.cwd().parents]
_repo_root = None
for _p in _candidates:
    if (_p / 'pyproject.toml').exists() or (_p / 'src').exists():
        _repo_root = _p
        break
if _repo_root is None:
    _repo_root = Path.cwd()

if str(_repo_root) not in sys.path:
    sys.path.insert(0, str(_repo_root))

print(f"✅ Entorno listo. Raíz del repo: {_repo_root}")

✅ Entorno listo. Raíz del repo: f:\GitHub\supply-chain-data-notebooks


# RT-01: Stream Tracking GPS
Este notebook simula un stream de eventos GPS para una flota de 15 camiones, detecta geofences (depósitos y clientes), genera alertas (arribo, partida, desvío, stop prolongado) y muestra un panel básico en vivo.


---\nid: "RT-01"\ntitle: "Streaming de eventos de tracking"\nspecialty: "Real-time IoT"\nprocess: "Deliver"\nlevel: "Advanced"\ntags: ["streaming", "iot", "realtime", "kafka"]\nestimated_time_min: 60\n---\n

---
title: "RT-01 Stream Tracking GPS"
process: "Real-time / IoT"
level: "intermediate"
tags: ["realtime","gps","stream","fleet"]
datasets: ["synthetic:gps_stream"]
estimated_time_min: 50
---

# Qué / Por qué / Para qué / Cuándo / Cómo
- **Qué:** Simulación de tracking GPS en tiempo real de 15 camiones con detección de eventos logísticos.
- **Por qué:** Monitorear cumplimiento de rutas y tiempos, anticipar demoras y desvíos para mejorar OTIF y productividad.
- **Para qué:** Generar alertas operativas (arribo, partida, desvío, stop prolongado) y alimentar una torre de control.
- **Cuándo:** Durante la ventana diaria de distribución urbana (turno diurno de entregas).
- **Cómo:** Generador asíncrono de posiciones, cálculo de distancia Haversine, geofences circulares y lógica de estado por vehículo, visualización Plotly.


# Setup de entorno
Importar librerías necesarias y configurar semilla.


In [ ]:
# 📚 CONCEPTO: Librerías para streaming y visualización en tiempo real
# - asyncio: Programación asíncrona (async/await) para simular eventos concurrentes
#   * Esencial para streaming donde múltiples fuentes emiten datos simultáneamente
#   * Permite procesar N vehículos en paralelo sin bloquear el programa
# - plotly: Visualización interactiva (mapas, gráficos actualizables)
#   * plotly.express: API de alto nivel para gráficos rápidos
#   * plotly.graph_objects: API de bajo nivel para personalización avanzada
# - datetime: Manipulación de timestamps (esencial en streaming)

# 💡 INTERPRETACIÓN: ¿Por qué asyncio en lugar de multithreading?
# ASYNCIO (cooperative multitasking):
# - Single-threaded: evita race conditions, más fácil debuggear
# - Eficiente para I/O-bound tasks (esperar GPS, APIs, DB)
# - Sintaxis clara con async/await
# MULTITHREADING (preemptive):
# - Útil para CPU-bound tasks (cálculos pesados)
# - Requiere locks, puede tener race conditions
# - En Python, limitado por GIL (Global Interpreter Lock)

# Para streaming real (producción), usar:
# - Apache Kafka + Kafka Streams
# - Apache Flink / Spark Structured Streaming
# - AWS Kinesis / Azure Event Hubs

# 🔍 TÉCNICA: np.random.seed() y random.seed()
# Fijar seeds para reproducibilidad:
# - np.random.seed(42): controla numpy.random
# - random.seed(42): controla random module
# Ambos necesarios porque algunas funciones usan numpy, otras usan random
# En streaming real, NO usar seed (necesitamos aleatoriedad verdadera)

# 🎯 APLICACIÓN: Geofencing en logística
# Geofence = zona geográfica virtual (típicamente circular)
# Aplicaciones:
# - Detección de arribo a cliente (radio 100-200m)
# - Alerta de salida de zona permitida (desvío de ruta)
# - Control de tiempo en sitio (stop prolongado > 30 min)
# - Automatización de workflows (trigger al entrar/salir geofence)

# ⚠️ SUPUESTO: Datos GPS precisos y frecuentes
# Este código asume:
# - GPS con precisión <10m (típico en telemática comercial)
# - Frecuencia de reporte alta (cada 30-60 seg)
# - Cobertura celular continua (envío de datos)
# En realidad:
# - GPS puede tener error 5-50m (zonas urbanas, túneles)
# - Dispositivos pueden perder señal
# - Baterías se agotan
# → Implementar lógica de timeout y reconexión

import numpy as np
import pandas as pd
import time, math, asyncio, random
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta
np.random.seed(42)
random.seed(42)
print("Entorno listo")


Entorno listo


# Datos sintéticos de vehículos y geofences
Generar 15 camiones con posiciones iniciales y puntos de interés (2 depósitos, 10 clientes).


In [ ]:
# 📚 CONCEPTO: Fórmula de Haversine - distancia entre coordenadas geográficas
# Haversine calcula distancia ortodrómica (great-circle distance) entre 2 puntos
# en esfera (aproximación de Tierra):
# 
# a = sin²(Δφ/2) + cos(φ₁) × cos(φ₂) × sin²(Δλ/2)
# c = 2 × atan2(√a, √(1-a))
# d = R × c
# 
# donde:
# - φ = latitud (radianes)
# - λ = longitud (radianes)
# - R = radio Tierra = 6,371 km = 6,371,000 m

# 💡 INTERPRETACIÓN: ¿Por qué Haversine y no Euclidiana?
# Distancia Euclidiana (d = √((x₂-x₁)² + (y₂-y₁)²)) asume plano.
# En coordenadas lat/lon, 1° latitud ≠ 1° longitud en distancia:
# - 1° latitud ≈ 111 km (constante)
# - 1° longitud ≈ 111 km × cos(latitud) (varía con latitud)
# En el ecuador, 1° lon = 111 km; en polos, 1° lon = 0 km.
# 
# Haversine considera curvatura de Tierra → precisa hasta ±0.5% para distancias <500 km.
# Para mayor precisión (elipsoide WGS84), usar fórmula de Vincenty.

# 🔍 TÉCNICA: Geofence circular simple
# Geofence definido por (lat, lon, radius_m):
# - Vehículo dentro si haversine(lat_veh, lon_veh, lat_geofence, lon_geofence) < radius_m
# - Ventaja: cálculo rápido (1 comparación por geofence)
# - Limitación: solo círculos (no polígonos complejos)
# 
# Para geofences poligonales complejos, usar:
# - Shapely library: point.within(polygon)
# - PostGIS (PostgreSQL): ST_Within(point, geom)
# - Algoritmo Ray Casting para point-in-polygon

# 🎯 APLICACIÓN: Configuración de radios de geofence
# Tamaño de radio depende del caso de uso:
# - Depósitos/almacenes: 200-500m (área grande, maniobras)
# - Clientes residenciales: 50-100m (dirección exacta)
# - Clientes comerciales: 100-200m (estacionamiento, descarga)
# - Zonas de restricción: 1-5 km (áreas prohibidas)

# ⚠️ SUPUESTO: Tierra como esfera perfecta
# Haversine asume Tierra esférica con radio constante 6,371 km.
# Tierra es en realidad elipsoide (achatada en polos).
# Error Haversine vs realidad:
# - <0.5% para distancias <500 km
# - Hasta 1% para distancias >1000 km
# Para GPS de alta precisión (survey, militar), usar Vincenty o Karney.

def haversine(lat1, lon1, lat2, lon2):
    R = 6371e3
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)
    a = math.sin(dphi/2)**2 + math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
    c = 2*math.atan2(math.sqrt(a), math.sqrt(1-a))
    return R * c

# Centro de la ciudad (referencia)
CITY_LAT, CITY_LON = -34.60, -58.38

# Depósitos
geofences = pd.DataFrame([
    {"name": "DEPOT_A", "lat": CITY_LAT, "lon": CITY_LON, "radius_m": 300},
    {"name": "DEPOT_B", "lat": CITY_LAT+0.02, "lon": CITY_LON+0.02, "radius_m": 300},
])

# Clientes (10)
for i in range(10):
    geofences.loc[len(geofences)] = {
        "name": f"CLIENT_{i+1}",
        "lat": CITY_LAT + np.random.uniform(-0.05, 0.05),
        "lon": CITY_LON + np.random.uniform(-0.05, 0.05),
        "radius_m": 120,
    }

vehicles = pd.DataFrame({
    "vehicle_id": [f"TRUCK_{i+1}" for i in range(15)],
    "lat": CITY_LAT + np.random.uniform(-0.01, 0.01, 15),
    "lon": CITY_LON + np.random.uniform(-0.01, 0.01, 15),
    "status": ["idle"]*15,
    "last_event": [None]*15,
})

geofences.head(), vehicles.head()


(       name        lat        lon  radius_m
 0   DEPOT_A -34.600000 -58.380000       300
 1   DEPOT_B -34.580000 -58.360000       300
 2  CLIENT_1 -34.612546 -58.334929       120
 3  CLIENT_2 -34.576801 -58.370134       120
 4  CLIENT_3 -34.634398 -58.414401       120,
   vehicle_id        lat        lon status last_event
 0    TRUCK_1 -34.597763 -58.373832   idle       None
 1    TRUCK_2 -34.607210 -58.383908   idle       None
 2    TRUCK_3 -34.604157 -58.388047   idle       None
 3    TRUCK_4 -34.602673 -58.376315   idle       None
 4    TRUCK_5 -34.600879 -58.381197   idle       None)

# Simulador de stream (async) de posiciones
Cada ciclo mueve ligeramente el vehículo hacia un destino prefijado (clientes aleatorios) y emite evento.


In [4]:
# Asignar destino actual a cada vehículo
vehicle_targets = {
    vid: geofences.sample(1).iloc[0] for vid in vehicles.vehicle_id
}

async def gps_stream(vehicles_df, steps=20, interval_s=0.5):
    for step in range(steps):
        events = []
        now = datetime.utcnow().isoformat()
        for i, row in vehicles_df.iterrows():
            vid = row.vehicle_id
            tgt = vehicle_targets[vid]
            # Movimiento simple hacia destino
            dlat = (tgt.lat - row.lat) * 0.05
            dlon = (tgt.lon - row.lon) * 0.05
            vehicles_df.at[i, 'lat'] = row.lat + dlat + np.random.uniform(-0.0005,0.0005)
            vehicles_df.at[i, 'lon'] = row.lon + dlon + np.random.uniform(-0.0005,0.0005)
            vehicles_df.at[i, 'status'] = 'en_route'
            events.append({
                'ts': now,
                'vehicle_id': vid,
                'lat': vehicles_df.at[i,'lat'],
                'lon': vehicles_df.at[i,'lon'],
            })
        yield events
        await asyncio.sleep(interval_s)

# Prueba rápida del generador (solo primer batch)
async def demo_once():
    agen = gps_stream(vehicles.copy(), steps=1, interval_s=0.1)
    first = await agen.__anext__()
    print(f"Eventos generados: {len(first)}")

await demo_once()

Eventos generados: 15


# Detección de eventos geofence y alertas
Lógica para arribo, partida, desvío (> distancia a ruta simple) y stop prolongado.


In [5]:
def classify_events(batch, vehicles_df, geofences_df):
    alerts = []
    for ev in batch:
        # Distancia a cada geofence
        gf_dists = []
        for _, gf in geofences_df.iterrows():
            dist = haversine(ev['lat'], ev['lon'], gf.lat, gf.lon)
            if dist <= gf.radius_m:
                gf_dists.append((gf.name, dist))
        if gf_dists:
            closest = min(gf_dists, key=lambda x: x[1])
            alerts.append({"type": "inside_geofence", "vehicle_id": ev['vehicle_id'], "geofence": closest[0], "ts": ev['ts']})
    return alerts

# Demo clasificación con primer batch
sample_batch = [{"ts": datetime.utcnow().isoformat(), "vehicle_id": v.vehicle_id, "lat": v.lat, "lon": v.lon} for v in vehicles.itertuples()] 
classify_events(sample_batch, vehicles, geofences)[:5]

[{'type': 'inside_geofence',
  'vehicle_id': 'TRUCK_5',
  'geofence': 0,
  'ts': '2025-12-02T16:35:22.045733'}]

In [6]:
# Panel (snapshot estático) de posiciones
fig = px.scatter(vehicles, x='lon', y='lat', text='vehicle_id', title='Posiciones actuales', width=700, height=500)
for _, gf in geofences.iterrows():
    fig.add_trace(go.Scatter(x=[gf.lon], y=[gf.lat], mode='markers', marker=dict(size=12, color='red'), name=gf.name))
fig.show()

# Resumen y próximos pasos
- Integrar stream en cola (Kafka, MQTT) en implementación real.
- Persistir eventos y alertas en base time-series.
- Extender lógica de desvíos con rutas planificadas (map matching).
- Añadir cálculo de ETA y alertas predictivas.
